[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/fast/fast_stable_diffusion.ipynb)

https://github.com/TheLastBen/fast-stable-diffusion

https://github.com/camenduru/$sdw-colab

https://github.com/Van-wise/sd-colab


**如果遇到报错,请联系up：**

https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244

In [ ]:
#@title 👇 1.启动SDwebui！
import os
import sys
import base64
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

sdw = base64.b64decode(("c3RhYmxlLWRpZmZ1c2lvbi13ZWJ1aQ==").encode('ascii')).decode('ascii')
w = base64.b64decode(("c2Qtd2VidWk=").encode('ascii')).decode('ascii')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'

#Check GPU  ,不使用GPU会出现莫名其妙的错误哦。
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
    raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

#克隆
%cd /content   
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  !git clone -q --branch master https://github.com/AUTOMATIC1111/$sdw
  %cd {webui_dir}
  !git reset --hard && git checkout master && rm webui.sh && git pull
  !mkdir -p {webui_dir}/cache
  os.environ['TRANSFORMERS_CACHE']=f"{webui_dir}/cache"

clear_output()
inf('\u2714 Done','success', '50px')

#安装环境&依赖 Requirements stable-diffusion-stability-ai
with capture.capture_output() as cap:
  %cd /content
  !mkdir -p {webui_dir}/repositories/stable-diffusion-stability-ai
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !tar -C /content --zstd -xf sd_rep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  !mv -n /content/sd/stablediffusion/* {webui_dir}/repositories/stable-diffusion-stability-ai && rm -rf /content/sd 
  
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  os.chdir('/content')

!apt -y install -qq aria2 git
!pip install -qq pycloudflared tomesd gradio
!pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv

clear_output()
inf('\u2714 Done','success', '50px')

#插件和配置部分放置云盘。
#中文插件&配置文件
!git clone https://github.com/hanamizuki-ai/$sdw-localization-zh_Hans {webui_dir}/extensions/$sdw-localization-zh_Hans  # hanamizuki-ai 中文
%cd {webui_dir} 
!wget -P {webui_dir} -O "config.json" "https://raw.githubusercontent.com/Van-wise/sd-colab/main/config.json"
#!wget -P {webui_dir} -O "config.json" "https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/ui-config.json"

#必备插件
!git clone https://github.com/zanllp/$w-infinite-image-browsing {webui_dir}/extensions/$w-infinite-image-browsing # zanllp 图片浏览
!git clone https://github.com/Physton/$w-prompt-all-in-one {webui_dir}/extensions/$w-prompt-all-in-one # 提示词插件
!git clone https://github.com/etherealxx/batchlinks-webui {webui_dir}/extensions/batchlinks-webui # 模型文件批量下载
!git clone https://github.com/camenduru/$w-tunnels {webui_dir}/extensions/$w-tunnels  # 用于 -share -cloudflared
!git clone https://github.com/Mikubill/$w-controlnet {webui_dir}/extensions/$w-controlnet # controlnet
!git clone https://github.com/KohakuBlueleaf/a1111-$w-lycoris {webui_dir}/extensions/a1111-$w-lycoris  # lycoris
!git clone https://github.com/AUTOMATIC1111/$sdw-rembg {webui_dir}/extensions/$sdw-rembg    # 背景移除
#!git clone https://github.com/canisminor1990/$w-kitchen-theme {webui_dir}/extensions/$w-kitchen-theme # kitchen主题
!git clone https://github.com/thomasasfk/$w-aspect-ratio-helper {webui_dir}/extensions/$w-aspect-ratio-helper #尺寸锁定

#放大插件
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 {webui_dir}/extensions/multidiffusion-upscaler-for-automatic1111 # Tile VAE 放大
#!git clone https://github.com/pkuliyi2015/$w-stablesr {webui_dir}/extensions/$w-stablesr # stablesr 放大
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {gwebui_dir}/models/ESRGAN -o 4x-UltraSharp.pth

#链接 models
ckpt_dir = f"{webui_dir}/models/Stable-diffusion"
gckpt_dir = f"{gwebui_dir}/models/Stable-diffusion"
!mkdir -p {ckpt_dir} {gckpt_dir} && ln -sf {gckpt_dir} {ckpt_dir}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {ckpt_dir} -o chilloutmix_NiPrunedFp32Fix.safetensors

#链接 Lora
Lora_dir = f"{webui_dir}/models/Lora"
gLora_dir = f"{gwebui_dir}/models/Lora"
!mkdir -p {Lora_dir} {gLora_dir} && ln -sf {gLora_dir} {Lora_dir}
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {gLora_dir} -o ShojoVibev_11.safetensors
!git clone https://huggingface.co/embed/lora /content
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/models/Lora/positive {gwebui_dir}/models/Lora

#链接 VAE
VAE_dir = f"{webui_dir}/models/VAE"
gVAE_dir = f"{gwebui_dir}/models/VAE"
!mkdir -p {VAE_dir} {gVAE_dir} && ln -sf {gVAE_dir} {VAE_dir}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {VAE_dir} -o vae-ft-mse-840000-ema-pruned.safetensors

#链接 embeddings
embeddings_dir = f"{webui_dir}/embeddings" 
gembeddings_dir = f"{gwebui_dir}/embeddings" 
!mkdir -p {embeddings_dir} {gembeddings_dir} && ln -sf {gembeddings_dir} {embeddings_dir}
!git clone https://huggingface.co/embed/negative /content
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/embeddings/negative {gwebui_dir}/embeddings

#链接 hypernetwork

#链接 scripts
scripts_dir = f"{webui_dir}/scripts" 
gscripts_dir = f"{gwebui_dir}/scripts" 
!mkdir -p {scripts_dir} {gscripts_dir} && ln -sf {gscripts_dir} {scripts_dir}
!wget https://raw.githubusercontent.com/camenduru/$sdw-scripts/main/run_n_times.py -O {gwebui_dir}/scripts/run_n_times.py

#链接 outputs
!mkdir -p {gwebui_dir}/outputs && ln -sf {gwebui_dir}/outputs {webui_dir}/outputs

clear_output()
inf('\u2714 Model downloaded, using the trained model.', 'success', '350px')

#链接 ControlNet
cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"
!mkdir -p {cn_dir} {gcn_dir}  && ln -sf {gcn_dir} {cn_dir}

#@markdown 选择ControlNet：
#@markdown 
#@markdown 0：不需要使用ControlNet
#@markdown 
#@markdown 1：下载ControlNet 模型到 Colab
controlnet = 1 #@param {type:"slider", min:0, max:1, step:1}
if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    #ControlNet Model Download 
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
else:
    print("不使用 ControlNet、或已使用快捷方式。")

clear_output()
inf('\u2714 Done','success', '50px')

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  
    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py
    !sed -i 's@{webui_dir}/repositories/stable-diffusion-stability-ai@/content/{sdw}/repositories/stable-diffusion-stability-ai/@' {webui_dir}/modules/paths.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\"quicksettings\": OptionInfo(.*@"quicksettings": OptionInfo("sd_model_checkpoint,  sd_vae, CLIP_stop_at_last_layers, inpainting_mask_weight, initial_noise_multiplier", "Quicksettings list"),@' {webui_dir}/modules/shared.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark  "

!python {webui_dir}/webui.py $wise


In [ ]:
#@title 👇启动 Stable-Diffusion
from IPython.utils import capture
import sys

with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  
    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py
    !sed -i 's@{webui_dir}/repositories/stable-diffusion-stability-ai@/content/{sdw}/repositories/stable-diffusion-stability-ai/@' {webui_dir}/modules/paths.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\"quicksettings\": OptionInfo(.*@"quicksettings": OptionInfo("sd_model_checkpoint,  sd_vae, CLIP_stop_at_last_layers, inpainting_mask_weight, initial_noise_multiplier", "Quicksettings list"),@' {webui_dir}/modules/shared.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark  "
!python {webui_dir}/webui.py $wise


In [ ]:
#@title 👇 附加模型快捷方式！
#附加模型的快捷方式，也就是使用别人分享的模型。附加过就不需要再次运行了！
import os
import sys
import base64

sdw = base64.b64decode(("c3RhYmxlLWRpZmZ1c2lvbi13ZWJ1aQ==").encode('ascii')).decode('ascii')
w = base64.b64decode(("c2Qtd2VidWk=").encode('ascii')).decode('ascii')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
!rm -rf {gwebui_dir}/$sdw
# get save_dir_id, save_dir = save_dir_id
def get_save_dir_id (save_dir):

    save_dir_id = None
    try:
        path_parts = save_dir.split('/')
        parent_id = None
        for folder_name in path_parts[4:]:
            query = f"mimeType='application/vnd.google-apps.folder' and trashed = false and name='{folder_name}'"
            if parent_id:
                query += f" and '{parent_id}' in parents"
            results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
            folder = results.get('files', [])[0]
            save_dir_id = folder.get('id')
            parent_id = save_dir_id
    except HttpError as error:
        print(f'An error occurred: {error}')
    print(save_dir_id)
    return save_dir_id

# /models/Stable-diffusion/sdmodels
def add_shortcut_directory_to_drive (shortcut_name, share_id, save_dir_name, save_dir):

    query = f"name='{save_dir_name}' and mimeType='application/vnd.google-apps.folder' and parents in '{save_dir}'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    if not results:
        raise Exception(f"No folder found with name '{save_dir_name}'.")
    
    save_dir_id = results[0]['id']
    shortcut_metadata = {
        'name': shortcut_name,
        'mimeType': 'application/vnd.google-apps.shortcut',
        'shortcutDetails': {
            'targetId': share_id
        },
        'parents': [save_dir_id]
    }
    try:
        shortcut_file = drive_service.files().create(body=shortcut_metadata, fields='id').execute()
        shortcut_id = shortcut_file['id']
        print("Shortcut created successfully!")
    
    except HttpError as error:
        print("An error occurred: %s" % error)
        shortcut_id = None
    
    return shortcut_id

# /models/Stable-diffusion/*
def add_shortcut_files_to_drive (share_id, save_dir_name, save_dir):

    query = f"name = '{save_dir_name}' and mimeType = 'application/vnd.google-apps.folder' and parents in '{save_dir}'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    if not results:
        raise Exception(f"No folder found with name '{save_dir_name}'.")
    
    save_dir_id = results[0]['id']
    shortcut_id = None

    query = f"'{share_id}' in parents and mimeType != 'application/vnd.google-apps.folder'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    file_ids = [result['id'] for result in results]

    for file_id in file_ids:
        file = drive_service.files().get(fileId=file_id).execute()
        file_name = file['name']
        shortcut_metadata = {
            'name': file_name,
            'mimeType': 'application/vnd.google-apps.shortcut',
            'shortcutDetails': {
               'targetId': file_id
            },
            "parents": [save_dir_id]
        }
        shortcut_file = drive_service.files().create(body=shortcut_metadata).execute()
        shortcut_id = shortcut_file['id']
        print(f"Shortcut created for {file_name} with ID: {shortcut_id}")

# delete_shortcut /models/Stable-diffusion/*
def delete_shortcut_files(save_dir):

    folder_id = get_save_dir_id(save_dir)
    
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.shortcut'"
    results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
    shortcuts = results.get('files', [])
 
    if shortcuts:
        print(f"Found {len(shortcuts)} shortcuts in folder {folder_id}:")
        for shortcut in shortcuts:
            print(f"{shortcut['name']} ({shortcut['id']})")
        for shortcut in shortcuts:
            drive_service.files().delete(fileId=shortcut['id']).execute()
        print(f"Deleted {len(shortcuts)} shortcuts")
    else:
        print(f"No shortcuts found in folder {folder_id}")
#@markdown 
#@markdown 使用云端主模型 （By B站 邀月浮白）
shortcut_to_models = True #@param {type:"boolean"}

if shortcut_to_models and (not os.path.exists(gwebui_dir+"/models/Stable-diffusion"+ "/sdmodels")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_directory_to_drive ("sdmodels", "16-eFzZcduNlu1c-ZHu9FE9FD4Jwi-6Er", "Stable-diffusion", save_dir)
  add_shortcut_directory_to_drive ("fbmodels", "1FuoMVN3GxaMP0J1mjA73SgmKWy_ELmXi", "Stable-diffusion", save_dir)

#@markdown 使用云端Lora模型 （By B站 邀月浮白）
shortcut_to_lora = True #@param {type:"boolean"}
if shortcut_to_lora and (not os.path.exists(gwebui_dir+"/models/Lora"+ "/fb_lora")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_directory_to_drive ("fb_lora", "1ZFNyj3LERv8W10TxR1zOGQVt0vTdl6x9", "Lora", save_dir)

#@markdown 使用云端VAE （By B站 邀月浮白）
shortcut_to_VAE = True #@param {type:"boolean"}
if shortcut_to_VAE and (not os.path.exists(gwebui_dir+"/models/VAE"+ "/fb_vae")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_files_to_drive ("1mpByHLWajydSg5iE223iVWRYfAhndAKA", "VAE", save_dir)

#@markdown 使用云端模ControlNet （By B站 邀月浮白）
shortcut_to_cn = True #@param {type:"boolean"}
if shortcut_to_cn = True
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(gcn_dir)
  delete_shortcut_files(save_dir)
  add_shortcut_files_to_drive ("1_sVbogR2El4FM-KvUf4YRNHCB1CpXvgO", "models", save_dir)

In [ ]:
#下载模型到 SDwebui （colab）
#下载主模型，请根据自己的需求下载，修改下载地址和文件名即可
!wget https://civitai.com/api/download/models/11745 -O {webui_dir}/models/Stable-diffusion/Chilloutmix-Ni-pruned-fp32-fix.safetensors

#下载 Lora 模型
!wget https://civitai.com/api/download/models/16557 -O {Lora_dir}/ShojoVibev_11.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {Lora_dir} -o ShojoVibev_11.safetensors

In [ ]:
#下载模型到 GoogleDrive
#下载链接，文件名，models路径
url = "https://civitai.com/api/download/models/16557" #models文件下载链接
filename = "ShojoVibev_11.safetensors" #保存文件名 .safetensors or .pt
save_dir = f"{gLora_dir}"  # f"{gLora_dir}/Stable-diffusion/"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o {filename} -d {save_dir} {url}